# Guest Session Feature Demo

This notebook is a demo of a feature that WhyLabs is working on called Guest Sessions. The feature builds ontop of our open source library, whylogs, to make it easier to visualize datasets and identify data issues. 

Typically when using whylogs, you'll log data to using one of the `whylogs.log()` apis to generate JSON and/or binary profiles locally that summarize various statistics about the dataset. That feature isn't going anywhere, don't worry! Guest Sessions take those generated statistics and uploads them to WhyLabs.ai to populate our dashboard without the need to sign in or share any personal identifiable information.

## How it works

The rest of the notebook will go through how the feature works and how to use it, starting with imports.

In [1]:
import pandas as pd
from datetime import datetime, timedelta
from whylogs.app.session import start_whylabs_session
from tqdm.auto import tqdm

### Load the data

Next, we'll load a sample dataset for logging with whylogs. The dataset was based on a set [from Kaggle](https://www.kaggle.com/yugagrawal95/sample-media-spends-data) for various advertising campaigns.

In [2]:
csv_file = "data/sample_media_spend.csv"
csv_dataframe = pd.read_csv(csv_file)
csv_dataframe

,Division,Days_In_Past,Paid_Views,Organic_Views,Google_Impressions,Email_Impressions,Facebook_Impressions,Affiliate_Impressions,Overall_Views,Sales
0,A,3,392,422,408,3.498950e+05,73580,12072,682,59417
1,N,3,720,254,928,8.089938e+05,116533,42869,318,128629
2,O,3,185,222,669,1.440437e+06,230891,63602,582,195068
3,X,3,642,852,200,3.787384e+05,57848,17072,473,62450
4,E,3,291,146,151,1.363087e+06,326747,83310,576,287095
...,...,...,...,...,...,...,...,...,...,...
346,S,1,38844,34276,1447904,2.533716e+06,95613,21997,72410,184597
347,H,1,18626,15887,397768,1.018171e+06,41344,8045,33672,68251
348,Z,1,52480,52321,1185817,2.675419e+06,71610,20145,104088,144794
349,O,1,84267,56717,2616351,4.106323e+06,158521,26292,140350,243933


### Group the data

The dataset is labeled with the number of days in the past (relative to today) the data happened for as a convenience for this demo. We'll group it up by that day offset.

In [3]:
grouped = csv_dataframe.groupby("Days_In_Past")

### Create a Guest Session

Next, we get to the main feature. We'll start a Guest Session with the `start_whylabs_session` api. The important thing about this API to notice is that there is a consent flag to data uploading.

```python
start_whylabs_session(data_collection_consent=True)
```

We just want to make it clear that this does upload statistical data to WhyLabs.ai. We don't ever upload raw data, only the profiles that we generate via logging. The flag has to be `True` or the feature will bail out.

In [4]:
session = start_whylabs_session(data_collection_consent=True, report_progress=True)

WARN: Missing config


### Log the data

Next we'll actually log the data and generate the statistical profiles. This will generate a profile for each day in the data.

In [5]:
for days_in_past, df in tqdm(grouped):
    timestamp = datetime.now() - timedelta(days=days_in_past)
    logger = session.logger(dataset_timestamp=timestamp)
    logger.log_dataframe(df)

  0%|          | 0/3 [00:00<?, ?it/s]

### Close the session and upload the generated profiles

Finally, we close the session which causes the profile uploads to WhyLabs.ai. The progress bar reflects the upload process. When it's done it will print out a URL that you can use to view the data in the WhyLabs.ai dashboard.

In [6]:
with session:
    print("done")

done


  0%|          | 0/3 [00:00<?, ?it/s]

You can explore your data in Observatory here: https://hub.whylabsapp.com/models/model-1/profiles/?sessionToken=session-74430


### View your data in WhyLabs

Clicking the URL above will bring you to the WhyLabs dashboard. You'll see a banner at the top that indicates that we may still be processing data in our backend. You'll land on our profile viewer page and you'll be able to see the uploaded profiles from here, possibly before all of the data finishes processing.